# Eigenvalue Optimization

Eigenvalue Optimization taken from: [GRANSO](http://www.timmitchell.com/software/GRANSO/) demo example 4.

## Problem Description

We have $M=A+BXC$,
where the matrices $A\in R^{N,N},B\in R^{N,M}$ and $C\in R^{P,N}$ are given, $X\in R^{M,P}$ is the matrix form optimization variable.

We have the nonconvex, nonsmooth, and constrained optimization problem
$$\min_{X}\max| \mathrm{Im} (\Lambda(A+BXC))|,$$
$$\text{s.t. }\alpha(A+BXC)+\xi \leq 0,$$
where $\mathrm{Im}(\cdot)$ is the imaginary part of complex number, $\xi$ is the stability margin, and $\Lambda(\cdot)$ is the spectrum of a square matrix $\cdot$, and $\alpha(\cdot)$ is the spectral abscissa of a square matrix, i.e., the maximum real part of its eigenvalues.

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct
import scipy.io
from torch import linalg as LA

## Data Initialization 
Specify torch device, and read the data from a provided file.

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device('cpu')

file = "./examples/data/spec_radius_opt_data.mat"
mat = scipy.io.loadmat(file)
mat_struct = mat['sys']
mat_struct = mat_struct[0,0]
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.from_numpy(mat_struct['A']).to(device=device, dtype=torch.double)
B = torch.from_numpy(mat_struct['B']).to(device=device, dtype=torch.double)
C = torch.from_numpy(mat_struct['C']).to(device=device, dtype=torch.double)
p = B.shape[1]
m = C.shape[0]
stability_margin = 1

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
# variables and corresponding dimensions.
var_in = {"X": [p,m] }

def user_fn(X_struct,A,B,C,stability_margin):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X

    # objective function
    M           = A + B@X@C
    [D,_]       = LA.eig(M)
    f = torch.max(D.imag)

    # inequality constraint, matrix form
    ci = pygransoStruct()
    ci.c1 = torch.max(D.real) + stability_margin

    # equality constraint 
    ce = None
    
    return [f,ci,ce]

comb_fn = lambda X_struct : user_fn(X_struct,A,B,C,stability_margin)

## User Options
Specify user-defined options for PyGRANSO

In [4]:
opts = pygransoStruct()
opts.torch_device = device
opts.maxit = 200
opts.x0 = torch.zeros(p*m,1).to(device=device, dtype=torch.double)
# print for every 10 iterations. default: 1
opts.print_frequency = 10

## Main Algorithm

In [5]:
start = time.time()
soln = pygranso(var_spec = var_in,combined_fn = comb_fn,user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## LBFGS 
(Optional) LBFGS and feasibility related options

In [6]:
opts = pygransoStruct()
opts.torch_device = device
opts.maxit = 200
opts.x0 = torch.zeros(p*m,1).to(device=device, dtype=torch.double)
# print for every 10 iterations. default: 1
opts.print_frequency = 10

# Limited-memory mode is generally not recommended for nonsmooth
# problems, such as this one, but it can nonetheless enabled if
# desired/necessary.  opts.limited_mem_size == 0 is off, that is, 
# limited-memory mode is disabled.
# Note that this example has 200 variables.
opts.limited_mem_size = 40

start = time.time()
soln = pygranso(var_spec = var_in,combined_fn = comb_fn,user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

/Users/jeffreyhu/Desktop/s25/csci-5527/pygranso/PyGRANSO/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3729.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


  10 ║ 1.000000 │  14.9749174293 ║  13.0237636352 ║ 1.396837 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.022802   ║ 
  20 ║ 1.000000 │  13.9063570499 ║  12.8289445410 ║ 1.037985 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.041164   ║ 
  30 ║ 0.387420 │  5.36505582158 ║  12.3734917629 ║ 0.755852 │   -  ║ S  │     2 │ 0.500000 ║     1 │ 0.009102   ║ 
  40 ║ 0.047101 │  0.78180422357 ║  12.4199883772 ║ 0.443629 │   -  ║ S  │     3 │ 0.250000 ║     1 │ 0.012637   ║ 
  50 ║ 0.047101 │  0.61703819957 ║  12.5364764038 ║ 0.162954 │   -  ║ S  │     9 │ 0.003906 ║     1 │ 0.001669   ║ 
  60 ║ 0.047101 │  0.58450664988 ║  12.3958134333 ║ 0.025454 │   -  ║ S  │     3 │ 0.250000 ║     1 │ 0.018595   ║ 
  70 ║ 0.047101 │  0.57534935458 ║  12.2135040112 ║ 0.008809 │   -  ║ S  │     2 │ 0.500000 ║     1 │ 0.005751   ║ 
  80 ║ 0.047101 │  0.57112933061 ║  12.1004585968 ║ 0.034383 │   -  ║ S  │     6 │ 0.093750 ║     1 │ 0.003038   ║ 
  90 ║ 0.047101 │  0.56668272611 ║  12.0302106758 ║ 0.006657 │   -  ║ S 

In [7]:
# We can also tune PyGRANSO to more aggressively favor satisfying
# feasibility over minimizing the objective.  Set feasibility_bias to
# true to adjust the following three steering parameters away from
# their default values.  For more details on these parameters, type
# import pygransoOptionsAdvanced
# help(pygransoOptionsAdvanced)
import numpy as np
opts = pygransoStruct()
opts.torch_device = device
feasibility_bias = True
if feasibility_bias:
    opts.steering_ineq_margin = np.inf    # default is 1e-6
    opts.steering_c_viol = 0.9         # default is 0.1
    opts.steering_c_mu = 0.1           # default is 0.9


In [8]:
opts.maxit = 200
opts.x0 = torch.zeros(p*m,1).to(device=device, dtype=torch.double)
# print for every 10 iterations. default: 1
opts.print_frequency = 10

start = time.time()
soln = pygranso(var_spec = var_in,combined_fn = comb_fn,user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

In my testing, with default parameters, PyGRANSO will first obtain a
feasible solution at iter ~= 160 and will reduce the objective to
11.60 by the time it attains max iteration count of 200.

With feasibility_bias = True, in my testing, PyGRANSO will obtain its
first feasible solution earlier, at iter ~= 60, but it will ultimately
have reduced the objective value less, only to 12.21, by the end of
its 200 maximum allowed iterations.